In [1]:
import pandas as pd 
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pymysql

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bandy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
db_connection = pymysql.connect(host='localhost', database='movie_reviews', user='bandytan', password='bt2102')

db_cursor = db_connection.cursor()

# movie1 = '10 Cloverfield Lane 2016'
# movie2 = '10 Things I Hate About You 1999'
# movie3 = '12 Angry Men 1957'

db_cursor.execute("SELECT * FROM review_data WHERE movie = '10 Cloverfield Lane 2016'")

table_rows = db_cursor.fetchall()

movie1 = pd.DataFrame(table_rows)

In [3]:
movie1.set_axis(['index', 'movie', 'rating', 'helpful', 'review'], axis=1, inplace=True)
movie1.isnull().values.any() #Check if data has any null values

False

In [4]:
movie1.head()

,index,movie,rating,helpful,review
0,400062,10 Cloverfield Lane 2016,0.333333,102,Unfortunately the ending ruined an otherwise ...
1,400063,10 Cloverfield Lane 2016,-0.111111,385,...oh dear Abrams. Again. I'll leave this rev...
2,400064,10 Cloverfield Lane 2016,-0.111111,110,"Fantastic, gripping, thoroughly enjoyable, un..."
3,400065,10 Cloverfield Lane 2016,-0.111111,73,"Hmmm... First of all, I must say that I was e..."
4,400066,10 Cloverfield Lane 2016,0.333333,42,Slow building & plot alternating claustrophob...


In [5]:
def process_review(column):
    column_processed = []
    for row in column:
        row_processed = re.sub(r'[^\w\s]', '', row) # remove punctuation
        column_processed.append(row_processed)
    return column_processed
movie1["review"] = process_review(movie1["review"])
movie1.shape

(728, 5)

In [6]:
def my_tokenizer(text):
    tokens = nltk.word_tokenize(text) # use the word_tokenize function from NLTK
    return tokens

In [17]:
def generate_nlp_features(df):
    vect = TfidfVectorizer( 
        tokenizer=my_tokenizer,
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    review = pd.Series(df["review"])

    tfidf_fit_review = vect.fit(review)
    tfidf_array = tfidf_fit_review.transform(review).toarray()
    tfidf_df = pd.DataFrame(tfidf_array)
    # print(tfidf_df.head())
    tfidf_df.columns = list(map(lambda x : "review_" + str(x), tfidf_df.columns))
    df = pd.merge(df, tfidf_df , left_index=True, right_index=True)

    feature_array = np.array(vect.get_feature_names())
    # print(feature_array)
    # print(sum(tfidf_array))

    # tfidf_fit = vect.fit(review)
    # tfidf_matrix = tfidf_fit.transform(review).toarray()
    feature_names = vect.get_feature_names()
    top_words = {}

    # # Get the indices of the top 50 words based on their TF-IDF scores
    top_word_indices = tfidf_array.sum(axis=0).argsort()[-50:][::-1]
    for i in top_word_indices:
        top_words[feature_names[i]] = tfidf_array.sum(axis=0)[i]
    # return top_words

    # tfidf_sorting = np.argsort(sum(tfidf_array)).flatten()[::-1]
    # print(tfidf_sorting)
    n = 100 # display 100 most important words
    # top_n = feature_array[tfidf_sorting][:n]
    # print(top_n)
    # tfidf_feature_names = list(zip(range(100), top_n))
    return (df, top_words)

In [18]:
df,  top_words= generate_nlp_features(movie1)
top_words

# scores = generate_nlp_features(movie1)
# for word, score in scores.items():
#     print(word, f'{score:.4f}')

C:\Users\bandy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\bandy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(
C:\Users\bandy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be remo

{'movie': 139.41791499863325,
 'film': 106.46005547941262,
 'cloverfield': 82.59030737126771,
 'good': 57.47150293747941,
 'one': 55.47202500926092,
 'ending': 55.4210523857393,
 'really': 54.38610067150412,
 'like': 52.438760177146115,
 'john': 51.26494961176741,
 '10': 50.775571217839364,
 'goodman': 48.86789558318817,
 'great': 46.70395292830538,
 'lane': 44.839818527318464,
 'cloverfield lane': 43.598872314161255,
 'would': 43.17749251882362,
 '10 cloverfield': 42.41964369969521,
 'end': 42.39500273158234,
 'story': 42.12218666798688,
 'first': 42.066519130384414,
 'well': 40.72398818836736,
 'howard': 40.57450591394839,
 'bunker': 40.53678066626006,
 '10 cloverfield lane': 40.45455610432843,
 'michelle': 38.90373093322014,
 'thriller': 36.70687019869485,
 'john goodman': 36.677918787377564,
 'time': 35.6739088384751,
 'dont': 35.35014757801402,
 'see': 34.17132103286348,
 'character': 34.06310504117336,
 'going': 33.71904522256483,
 'watch': 33.69423845318077,
 'much': 33.26859812

In [19]:
#Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement.
df["review_polarity"] = df["review"].apply(lambda x: TextBlob(x).sentiment.polarity)  
df.head()

,index,movie,rating,helpful,review,review_0,review_1,review_2,review_3,review_4,...,review_91,review_92,review_93,review_94,review_95,review_96,review_97,review_98,review_99,review_polarity
0,400062,10 Cloverfield Lane 2016,0.333333,102,Unfortunately the ending ruined an otherwise ...,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.151079,0.139286
1,400063,10 Cloverfield Lane 2016,-0.111111,385,oh dear Abrams Again Ill leave this review fa...,0.197767,0.054271,0.055388,0.0,0.000000,...,0.058923,0.0,0.06438,0.118798,0.054672,0.0,0.000000,0.0,0.164623,0.103823
2,400064,10 Cloverfield Lane 2016,-0.111111,110,Fantastic gripping thoroughly enjoyable until...,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.00000,0.000000,0.199729,0.0,0.141698,0.0,0.000000,0.054675
3,400065,10 Cloverfield Lane 2016,-0.111111,73,Hmmm First of all I must say that I was expec...,0.000000,0.000000,0.000000,0.0,0.140915,...,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.202911,0.024167
4,400066,10 Cloverfield Lane 2016,0.333333,42,Slow building plot alternating claustrophobi...,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.00000,0.128378,0.118161,0.0,0.000000,0.0,0.000000,0.077841


In [20]:
print(min(df.review_polarity))
print(max(df.review_polarity))

-0.9549999999999998
0.8


In [75]:
# Export feature names as txt
with open('movie3_tfidf_feature_names.txt', 'a') as f:
    f.write(str(tfidf_feature_names))

# Export dataframe as csv
df.to_csv("movie3.csv", index=False)
